In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import holoviews as hv
from holoviews import opts, dim
import pickle
from scipy import stats

In [ ]:
hv.extension("bokeh")

# Load data

In [ ]:
# with open('190311photobleaching.pickle', 'rb') as f:
# with open('all_data.pickle', 'rb') as f:
# with open('190313photobleaching_noflatcorr.pickle', 'rb') as f:
# with open('190326photobleaching_flatcorr_fluoronly.pickle', 'rb') as f:
with open("190405photobleaching_noflatcorr.pickle", "rb") as f:
    d = pickle.load(f)

In [ ]:
list(d.keys())

In [ ]:
measurements, regionprops, labels, img = d[
    "/n/scratch2/jqs1/fidelity/190313/fluorescence/190313_mkate_100ms_50pct_laser.nd2"
][0]

In [ ]:
# regionprops.reset_index(inplace=True)
regionprops.head()

In [ ]:
%%output size=150
#%%opts Image {+axiswise}
hv.Image(img / img.max()).options(cmap="gray") + hv.Image(labels != 0).options(
    cmap="blues"
)

# Plotting

In [ ]:
list(measurements.keys())

In [ ]:
traces = []
rp_list = []
rp_df = pd.DataFrame()
directory = "/n/scratch2/jqs1/fidelity/190326/190326_mkate_nochlor_wateragarpad_100ms_100pct_laser.nd2"
# directory =  '/n/scratch2/jqs1/fidelity/190313/fluorescence/190313_mkate_100ms_100pct_laser.nd2'
# directory = '/n/scratch2/jqs1/fidelity/190325/phase/SB1_mcherry_LBagarpad_100ms_100pct_laser/190313_SB1_mcherry_LBagarpad_100ms_100pct_laser.nd2'
for i in d[directory].keys():
    #     if i == 4:
    if i not in np.arange(9, 9).astype(list):
        print(i)
        #     measurements, regionprops, labels, img = d['/n/scratch2/jqs1/fidelity/190301/jqs_photobleach_100ms_de32_mkate2_000{:d}.nd2'.format(i)]
        regionprops = d[directory][i]["regionprops"]
        measurements = d[directory][i]["traces"]
        area = d[directory][i]["regionprops"]["area"]

        bg_trace = measurements["mean"][0, :]
        bg = np.nanmin(measurements["mean"][1:], axis=1)
        #         print(np.nanmin(measurements['mean'][1:],axis=1).shape)

        #         traces.append((measurements['mean'][1:] - 0*bg[:,np.newaxis])*area[:,np.newaxis])
        #         traces.append(measurements['mean'][1:] - bg_trace)
        traces.append(measurements["mean"][1:])

        rp_list.append(regionprops)
        print(measurements["mean"][1:].shape)
traces = np.concatenate(traces)
rp_df = pd.concat(rp_list, sort=False)
rp_df.reset_index(inplace=True)
print(traces.shape)
data = pd.DataFrame(traces)  # + np.random.normal(0,1,Gsamp.T.shape))

# bins = np.linspace(data[0].min(),data[0].max(),150)
data = pd.concat([data, rp_df], axis=1, sort=False)
# rp_df['initial_intensity'] = df[0]
# data['bin'] = np.zeros(len(traces))
x_bin = pd.cut(data["centroid_x"], 40)
y_bin = pd.cut(data["centroid_y"], 40)
xy_bin_df = pd.DataFrame({"x": x_bin.values.codes, "y": y_bin.values.codes})
data["bin"] = pd.Categorical(xy_bin_df.apply(lambda x: tuple(x.values), axis=1)).codes
# data['bin'] = pd.Categorical().values
# data['bin'] = np.arange(len(traces))

t_end = traces.shape[1] - 6
data["k"] = data.apply(
    lambda y: (y.values[0] - y.values[t_end - 1])
    / np.sum(y[:t_end].values - y.values[t_end - 1]),
    axis=1,
).values
# cfit = data.apply(lambda y: curve_fit(func1, np.arange(0,y[:t_end].shape[0]), y[:t_end],p0=[y[0],.001,0])[0], axis=1).apply(lambda x: pd.Series(x,index=['I0','k','bg']))
# data[:t_end] = data[:t_end].values - cfit.bg.values[:,np.newaxis]

In [ ]:
%%output size=250
# traces = measurements['mean']
idxs = np.random.permutation(traces.shape[0])
downsample = 100  # set to 1 to show all traces (instead odata.k.hist()f 10%); this will make your browser slow
curves = [
    {"x": np.arange(traces.shape[1]), "y": traces[i], "i": idxs[i]}
    for i in range(traces.shape[0] // downsample)
]
hv.Contours(curves, vdims=["i"]).options(color_index="i", cmap="Category20", logy=True)

In [ ]:
tpoint = 600
blorp = traces[traces[:, tpoint] < 100, tpoint]
# blorp = blorp[::10]
plt.hist(blorp, bins=30, density=True)
x = np.linspace(blorp.min(), blorp.max(), 100)
mu, sigma = stats.norm.fit(blorp)
print(mu, sigma**2)
print(stats.kstest(blorp, "norm", args=(mu, sigma)))
plt.plot(x, stats.norm.pdf(x, mu, sigma), linewidth=6, alpha=0.8)

In [ ]:
%matplotlib inline
import pylab as plt
import pandas as pd
import numpy as np
from matplotlib import cm
from scipy.integrate import simps, trapz
from astropy.stats.biweight import biweight_midvariance


def filter_df(df, prop_dict):
    processed_df = df.copy()
    for prop in prop_dict:
        processed_df = processed_df[
            (processed_df[prop] > prop_dict[prop][0])
            & (processed_df[prop] < prop_dict[prop][1])
        ]

    return processed_df


def get_stats(df, thresh):
    trajectories = df.iloc[:, :t_end]
    p = trajectories.apply(lambda x: x / x[0], axis=1)
    #     p['bin'] = df['bin']
    pbar = p.mean()
    mu = trajectories.mean()
    sigma2 = (p - pbar) ** 2
    #     sigma2 = (1.48*df.iloc[:,:t_end].groupby(df['bin']).mad()[df.groupby(df['bin']).size() > thresh])**2
    #     sigma2 = (df.iloc[:,:t_end].groupby(df['bin']).iqr()[df.groupby(df['bin']).size() > thresh]/1.35)**2
    #     sigma2 = sigma2 - (pbar**2).multiply(sigma2[0].values,axis=0)
    return pbar, mu, sigma2


def nu_int(pbar, mu, sigma, q=1):
    nu_dict = {}  # pd.Series()
    cq = -1 / (1 / 2 * q**2 - 1 / 3 * q**3)
    y = sigma2
    #     print(y)
    dp = pbar.values
    dp = dp[pbar.values > 1 - q]
    #     print(y.shape, )
    f = y.iloc[pbar.values > 1 - q]

    nu_dict[name] = cq * simps(f, dp)

    return pd.Series(nu_dict)


def fluct_plot(pbar, mu, sigma2, thresh, q=1):
    hist_df = df.groupby(df["bin"]).size()
    hist_df = hist_df[hist_df.values > thresh]
    #     print(hist_df)
    #     print(f'Total number of cells is {np.sum(hist_df.values)}')

    plt.figure(figsize=(12, 8))
    cmap = cm.get_cmap("coolwarm")

    y = sigma2 * mu[0]
    y[np.isnan(y)] = 0

    cq = -1 / (1 / 2 * q**2 - 1 / 3 * q**3)
    #     cq = -1/(q*(1/2 - (2/3)*q**2))

    nu = np.nanmean(
        cq * simps(y.T.iloc[pbar.values > 1 - q].T, pbar.iloc[pbar.values > 1 - q])
    )
    #     qind = (pbar.values > (1/2 - q)) & (pbar.values < (1/2 + q))
    #     print(cq,pbar.iloc[qind],y.T.iloc[qind].T)
    #     nu = np.nanmean(cq*simps(y.T.iloc[qind].T,pbar.iloc[qind]))

    #     c = (name.left - imin)/(imax - imin)
    plt.scatter(1 - pbar, np.nanmean(y, axis=0), color="k", label="Single-cell data")
    #     plt.fill_between(1-pbar,np.nanmean(y,axis=0),alpha=.1,color='r')

    p = np.linspace(0, 1, 1000)
    plt.plot(
        p,
        nu * p * (1 - p),
        linewidth=6,
        alpha=0.5,
        color="g",
        label="Theoretical curve",
    )
    p_left = np.linspace(0, q, 100)
    p_right = np.linspace(q, 1, 100)

    plt.fill_between(
        p_left,
        nu * p_left * (1 - p_left),
        alpha=0.3,
        color="c",
        label="Integrated area",
    )
    plt.fill_between(
        p_right,
        nu * p_right * (1 - p_right),
        alpha=0.3,
        color="r",
        hatch="/",
        label="Interpolated area",
    )

    plt.vlines(
        q, 0, nu * q * (1 - q), alpha=0.5, linewidth=5, color="m", label=f"q = {q}"
    )
    plt.title(
        rf"$\nu =  {-cq:.1f} \cdot \int\frac{{\hat{{\sigma}}^2}}{{f_{{max}}}}dp$ = {nu:.6f}",
        fontsize=20,
        pad=20,
    )
    plt.xlabel(r"$(1-\hat{p})$", fontsize=20)
    plt.ylabel(r"$\frac{\hat{\sigma}^2}{f_{max}}$", fontsize=20)
    plt.legend()


#     plt.ylim(bottom=0,top=.0004)
thresh = 50

prop_dict = {
    "centroid_x": [500, 1700],
    "centroid_y": [500, 1700],
    "area": [
        data.area.median() - 2 * data.area.std(),
        data.area.median() + 2 * data.area.std(),
    ],
    0: [500, 3000],
}
# prop_dict = {'area': [data.area.median() - 2*data.area.std(), data.area.median() + 2*data.area.std()],
#               0: [800, 3000]}

df = filter_df(data, prop_dict)
print(f"Total number of cells used is {df.shape[0]} out of {data.shape[0]}")
pbar, mu, sigma2 = get_stats(df, thresh)

q = 1 / 2
# nu_df = nu_int(pbar,mu,sigma2,q)
# print(nu_df)

fluct_plot(pbar, mu, sigma2, thresh, q)

In [ ]:
%matplotlib inline
import pylab as plt
import pandas as pd
import numpy as np
from matplotlib import cm
from scipy.integrate import simps, trapz
from astropy.stats.biweight import biweight_midvariance
from scipy.optimize import curve_fit
from scipy.optimize import least_squares
import seaborn as sns


def func1(x, a, b, c):
    return a * np.exp(-b * x) + c


def func2(x, t, y):
    return x[0] * np.exp(-x[1] * t) - y


def filter_df(df, prop_dict):
    processed_df = df.copy()
    for prop in prop_dict:
        processed_df = processed_df[
            (processed_df[prop] > prop_dict[prop][0])
            & (processed_df[prop] < prop_dict[prop][1])
        ]

    return processed_df


def tau_interp(y, t, kbar, k):
    #     k = curve_fit(func, t, y)[0][1]
    #     k = np.polyfit(t[0:600],np.log(y[0:600]),1)[0]
    #     k = np.mean((y[:-1] - y[1:])/(y[:-1]))
    #     print(k,kbar)

    tau = (kbar / k * t).astype(np.int)
    tau = tau[tau < y.shape[0]]
    #     print(k)
    y_tau = y[tau]
    if y.shape[0] > tau.shape[0]:
        y_tau = np.pad(
            y_tau, (0, t.shape[0] - tau.shape[0]), "constant", constant_values=np.nan
        )

    #     print(type(y_tau))
    #     y_tau = y*np.exp((k-kbar)*t)
    return row


def nl_correct(row):
    tpoints = np.arange(0, t_end)
    p, _ = curve_fit(func1, tpoints, row, p0=[row[0] - row.min(), 0.001, row.min()])
    #     return pd.Series(func1(tpoints,p[0],p[1],0))
    #     print(p[1])
    return (row - p[2]) * p[0] / row.iloc[0]


def get_stats(df):

    #     df_agg = df.groupby('bin').mean()
    #     k_agg = df_agg.apply(lambda y: (y.values[0] - y.values[t_end-1])/np.sum(y[:t_end].values),axis=1)
    #     df['k_agg'] = k_agg[df['bin']].values
    #     y = df.apply(lambda f: tau_interp(f.values[:t_end],np.arange(t_end),np.nanmean(k_agg),f['k_agg']),axis=1)

    y = df.iloc[:, :t_end]
    #     y = y - 1.13*y.min(axis=1).mean()#[:,np.newaxis]
    y = y.apply(nl_correct, axis=1)
    print(y.shape)
    p = pd.DataFrame(y.values / y[0].values[:, np.newaxis])

    pbar = p.mean()
    mu = y.mean()
    sigma2 = (p - pbar) ** 2
    #     sigma2 -= sigma2[0]*pbar**2
    #     sigma2 = y.var() - y[0].var()*pbar**2

    #     sigma2 = pd.DataFrame(sigma2.values*y[0].values[:,np.newaxis])
    return y, pbar, mu, sigma2


def nu_int(pbar, mu, sigma, q=1):
    nu_dict = {}  # pd.Series()
    cq = -1 / (1 / 2 * q**2 - 1 / 3 * q**3)
    y = sigma2
    #     print(y)
    dp = pbar.values
    dp = dp[pbar.values > 1 - q]
    #     print(y.shape, )
    f = y.iloc[pbar.values > 1 - q]

    nu_dict[name] = cq * simps(f, dp)

    return pd.Series(nu_dict)


def fluct_plot(pbar, mu, sigma2, y0, q=1):
    #     print(hist_df)
    #     print(f'Total number of cells is {np.sum(hist_df.values)}')

    plt.figure(figsize=(12, 8))
    cmap = cm.get_cmap("coolwarm")
    y = pd.DataFrame(sigma2.values * y0.values[:, np.newaxis])
    #     y -= y.iloc[1]*pbar**2
    y = y.iloc[:, 1:]
    pbar = pbar.iloc[1:]
    ybar = y.mean(axis=0)
    ybar -= ybar.iloc[0] * pbar**2
    print(y.shape, pbar.shape)
    #     y = sigma2*mu[0]
    y[np.isnan(y)] = 0

    cq = -1 / (1 / 2 * q**2 - 1 / 3 * q**3)
    #     cq = -1/(q*(1/2 - (2/3)*q**2))

    #     nu = np.nanmean(cq*simps(y.T.iloc[pbar.values > 1-q].T,pbar.iloc[pbar.values > 1-q]))
    nu = cq * simps(ybar.T.iloc[pbar.values > 1 - q].T, pbar.iloc[pbar.values > 1 - q])
    #     qind = (pbar.values > (1/2 - q)) & (pbar.values < (1/2 + q))
    #     print(cq,pbar.iloc[qind],y.T.iloc[qind].T)
    #     nu = np.nanmean(cq*simps(y.T.iloc[qind].T,pbar.iloc[qind]))

    #     c = (name.left - imin)/(imax - imin)
    plt.scatter(1 - pbar, ybar, color="k", label="Single-cell data")
    #     plt.fill_between(1-pbar,np.nanmean(y,axis=0),alpha=.1,color='r')

    p = np.linspace(0, 1, 1000)
    plt.plot(
        p,
        nu * p * (1 - p),
        linewidth=6,
        alpha=0.5,
        color="g",
        label="Theoretical curve",
    )
    p_left = np.linspace(0, q, 100)
    p_right = np.linspace(q, 1, 100)

    plt.fill_between(
        p_left,
        nu * p_left * (1 - p_left),
        alpha=0.3,
        color="c",
        label="Integrated area",
    )
    plt.fill_between(
        p_right,
        nu * p_right * (1 - p_right),
        alpha=0.3,
        color="r",
        hatch="/",
        label="Interpolated area",
    )

    plt.vlines(
        q, 0, nu * q * (1 - q), alpha=0.5, linewidth=5, color="m", label=f"q = {q}"
    )
    plt.title(
        rf"$\nu =  {-cq:.1f} \cdot \int\frac{{\hat{{\sigma}}^2}}{{f_{{max}}}}dp$ = {nu:.6f}",
        fontsize=20,
        pad=20,
    )
    plt.xlabel(r"$(1-\hat{p})$", fontsize=20)
    plt.ylabel(r"$\frac{\hat{\sigma}^2}{f_{max}}$", fontsize=20)
    plt.legend()


#     plt.ylim(bottom=0,top=.0004)


def kmaps(df):
    fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(24, 12))

    cmap = cm.get_cmap("coolwarm")

    fit_df = df[["centroid_x", "centroid_y", "k", 0]].copy()
    fit_df["c_k"] = (fit_df.k - fit_df.k.min()) / (fit_df.k.max() - fit_df.k.min())
    fit_df["c_I0"] = (fit_df[0] - fit_df[0].min()) / (fit_df[0].max() - fit_df[0].min())

    sns.regplot(
        data=fit_df,
        x="centroid_x",
        y="centroid_y",
        fit_reg=False,
        ax=axs[0],
        scatter_kws={"color": cmap(fit_df["c_k"])},
    )
    sns.regplot(
        data=fit_df,
        x="centroid_x",
        y="centroid_y",
        fit_reg=False,
        ax=axs[1],
        scatter_kws={"color": cmap(fit_df["c_I0"])},
    )

    axs[0].set_xlim([0, 2000])
    axs[0].set_ylim([0, 2000])
    axs[1].set_xlim([0, 2000])
    axs[1].set_ylim([0, 2000])

In [ ]:
import time

prop_dict = {
    "area": [
        data.area.median() - 2 * data.area.std(),
        data.area.median() + 2 * data.area.std(),
    ],
    0: [1000, 2500],
    t_end - 1: [0, 400],
    "k": [0.0075, 0.009],
}
# prop_dict = {'area': [data.area.median() - 2*data.area.std(), data.area.median() + 2*data.area.std()],
#              'k': [.0038,.004],
#               0: [4e4,1e5]}
# prop_dict = {'area': [data.area.median() - 2*data.area.std(), data.area.median() + 2*data.area.std()],
#               0: [1400, 2500], t_end-1: [50,300],
#              'k': [.007,.0085]}
# prop_dict = {'centroid_x': [850, 1250],
#              'centroid_y': [750, 1150],
#              'area': [data.area.median() - 2*data.area.std(), data.area.median() + 2*data.area.std()],
#              0: [500, 3000]}

df = filter_df(data, prop_dict)
df.k.hist(bins=30)


y, pbar, mu, sigma2 = get_stats(df)


plt.figure(figsize=(12, 8))
plt.semilogy(y.T.values[:, ::20] + 1)
plt.ylim(bottom=1)

q = 0.7


fluct_plot(pbar, mu, sigma2, y[0], q)

kmaps(df)

In [ ]:
from scipy.stats import linregress
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

# plt.figure(figsize=(12,8))

# ax = fig.add_subplot(111, projection='3d')
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(24, 12))

cmap = cm.get_cmap("coolwarm")


fit_df = df[["centroid_x", "centroid_y", "k", 0]].copy()
# fit_df = fit_df[fit_df[0] > 1000]
# fit_df = fit_df[fit_df['k'] < .003]

fit_df["c_k"] = (fit_df.k - fit_df.k.min()) / (fit_df.k.max() - fit_df.k.min())
fit_df["c_I0"] = (fit_df[0] - fit_df[0].min()) / (fit_df[0].max() - fit_df[0].min())

print("plotting!")
sns.regplot(
    data=fit_df,
    x="centroid_x",
    y="centroid_y",
    fit_reg=False,
    ax=axs[0],
    scatter_kws={"color": cmap(fit_df["c_k"])},
)
sns.regplot(
    data=fit_df,
    x="centroid_x",
    y="centroid_y",
    fit_reg=False,
    ax=axs[1],
    scatter_kws={"color": cmap(fit_df["c_I0"])},
)


# sns.plt.show()

# for x, y, c_k, c_I0 in zip(fit_df.centroid_x, fit_df.centroid_y, fit_df.c_k, fit_df.c_I0):
# #     print(x,y,c)
#     axs[0].scatter(x,y,color=cmap(c_k))
#     axs[1].scatter(x,y,color=cmap(c_I0))

axs[0].set_xlim([0, 2000])
axs[0].set_ylim([0, 2000])
axs[1].set_xlim([0, 2000])
axs[1].set_ylim([0, 2000])

# m_list = []
# for i in range(traces.shape[0]):
#     y = np.log10(traces[i]-bg_trace)

#     m, b = np.polyfit(x[:-3],y[:-3],1)
#     m_list.append(m)
# # plt.plot(x,y)
# plt.hist(m_list,bins=30)
# print(np.mean(m_list))

In [ ]:
%matplotlib inline
import pylab as plt
import pandas as pd
import numpy as np
from matplotlib import cm
from scipy.integrate import simps
from astropy.stats.biweight import biweight_midvariance


def filter_df(df, prop_dict):
    processed_df = df.copy()
    for prop in prop_dict:
        processed_df = processed_df[
            (processed_df[prop] > prop_dict[prop][0])
            & (processed_df[prop] < prop_dict[prop][1])
        ]

    return processed_df


def get_stats(df, thresh):
    trajectories = df.iloc[:, :t_end]
    p = trajectories.apply(lambda x: x / x[0], axis=1)
    p["bin"] = df["bin"]
    pbar = p.groupby(p["bin"]).mean()[p.groupby(p["bin"]).size() > thresh]
    mu = trajectories.groupby(df["bin"]).mean()[df.groupby(df["bin"]).size() > thresh]
    sigma2 = trajectories.groupby(df["bin"]).var(ddof=0)[
        df.groupby(df["bin"]).size() > thresh
    ]
    #     sigma2 = (1.48*df.iloc[:,:t_end].groupby(df['bin']).mad()[df.groupby(df['bin']).size() > thresh])**2
    #     sigma2 = (df.iloc[:,:t_end].groupby(df['bin']).iqr()[df.groupby(df['bin']).size() > thresh]/1.35)**2
    #     sigma2 = sigma2 - (pbar**2).multiply(sigma2[0].values,axis=0)
    return pbar, mu, sigma2


def nu_int(pbar, mu, sigma, q=1):
    nu_dict = {}  # pd.Series()
    cq = -1 / (1 / 2 * q**2 - 1 / 3 * q**3)
    y = sigma2.div(mu[0].values, axis="rows")
    for name, group in pbar.groupby("bin"):
        dp = pbar.loc[name].values
        dp = dp[pbar.loc[name].values > 1 - q]
        f = y.loc[name].values[pbar.loc[name].values > 1 - q]
        nu_dict[name] = cq * simps(f, dp)

    return pd.Series(nu_dict)


def fluct_plot(pbar, mu, sigma2, thresh, q=1):
    hist_df = df.groupby(df["bin"]).size()
    hist_df = hist_df[hist_df.values > thresh]
    print(hist_df)
    print(f"Total number of cells is {np.sum(hist_df.values)}")

    plt.figure(figsize=(12, 8))
    cmap = cm.get_cmap("coolwarm")

    y = sigma2.div(mu[0].values, axis="rows")
    imax = pbar.index[-1].left
    imin = pbar.index[0].left
    q = 1
    cq = -1 / (1 / 2 * q**2 - 1 / 3 * q**3)
    # plt.vlines(q,0,3)

    for name, group in pbar.groupby("bin"):
        c = (name.left - imin) / (imax - imin)
        plt.scatter(
            1 - pbar.loc[name].values,
            y.loc[name].values,
            color=cmap(c),
            label=name.left,
        )
        plt.legend()

    p = np.linspace(0, 1, 1000)
    plt.plot(p, nu_df.mean() * p * (1 - p), linewidth=6, alpha=0.8, color="g")

    plt.title(
        rf"$\nu =  {-cq:.0f} \cdot \int\frac{{\hat{{\sigma}}^2}}{{f_{{max}}}}dp$ = {nu_df.mean():.2f}",
        fontsize=20,
        pad=20,
    )
    plt.xlabel(r"$(1-\hat{p})$", fontsize=20)
    plt.ylabel(r"$\frac{\hat{\sigma}^2}{f_{max}}$", fontsize=20)


#     plt.ylim([0,.0001])
thresh = 100

# prop_dict = {'centroid_x': [500, 1700],
#              'centroid_y': [500, 1700],
#              'area': [data.area.median() - 2*data.area.std(), data.area.median() + 2*data.area.std()],
#              0: [800, 3000]}
prop_dict = {
    "area": [
        data.area.median() - 2 * data.area.std(),
        data.area.median() + 2 * data.area.std(),
    ],
    0: [800, 3000],
}

df = filter_df(data, prop_dict)
pbar, mu, sigma2 = get_stats(df, thresh)

q = 1
nu_df = nu_int(pbar, mu, sigma2, q)
print(nu_df)

fluct_plot(pbar, mu, sigma2, thresh, q)

In [ ]:
%%output size=250
# traces = measurements['mean']
idxs = np.random.permutation(traces.shape[0])
downsample = (
    10  # set to 1 to show all traces (instead of 10%); this will make your browser slow
)
curves = [
    {"x": np.arange(traces.shape[1]), "y": df[i], "i": idxs[i]}
    for i in range(traces.shape[0] // downsample)
]
hv.Contours(curves, vdims=["i"]).options(color_index="i", cmap="Category20", logy=True)

In [ ]:
I0 = traces[:, 0]
plt.hist(I0, bins=30)
print(np.mean(I0), np.std(I0))

In [ ]:
cmap = cm.get_cmap("coolwarm")
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(24, 12))
gridfig, gridaxs = plt.subplots(
    nrows=int(np.ceil(nu_df.size / 4)), ncols=4, figsize=(24, 30)
)
imax = df.bin.values[-1].left
imin = df.bin.values[0].left
ind = 0
for name, group in df.groupby("bin"):
    if group.shape[0] > thresh:  # & (np.round(name.left) == 3352):
        #         c = .5
        c = (name.left - imin) / (imax - imin)
        #         c = (nu_df[name] - nu_df.min())/(nu_df.max() - nu_df.min())
        #         z = name.left*np.ones(group.centroid_x.shape)

        axs[0].scatter(
            group.centroid_x, group.centroid_y, color=cmap(c), marker="o", s=16
        )  # , label=nu_df[name])
        axs[0].set_xlim(data.centroid_x.min(), data.centroid_x.max())
        axs[0].set_ylim(data.centroid_y.min(), data.centroid_y.max())
        axs[1].semilogy(group.iloc[:, :t_end].T, color=cmap(c))
        axs[1].set_ylim([1, 3000])
        #         tempfig = plt.figure(figsize=(8,8))
        gridaxs.flatten()[ind].semilogy(group.iloc[:, :t_end].T, color=cmap(c))
        #         gridaxs.flatten()[ind].set_title(rf'Interval = {str(name)}, $\nu = $ {nu_df[name]:.2f}')
        gridaxs.flatten()[ind].set_ylim([1, 2000])
        ind += 1
    else:
        axs[0].scatter(
            group.centroid_x, group.centroid_y, color="k", marker="x", s=12, alpha=0.3
        )
gridfig.tight_layout()